In [1]:
#必要なライブラリをインポート
import pandas as pd
import codecs
import numpy as np

In [2]:
#列をたくさん表示するため
pd.set_option('display.max_columns', None)

#ファイルの読み込み
with codecs.open("/home/uk/all_data/origin_data.csv", "r", "Shift-JIS", "ignore") as file:
    df = pd.read_csv(file, delimiter=",", names=["年","月","日","馬名","馬番","枠番","年齢","性別","馬体重","斤量","場所","頭数","距離","馬場状態","天候","人気","単勝オッズ","確定着順","タイムS","着差タイム","トラックコード"])

#欠損値NaNが存在する行の削除
df = df.dropna()

#削除した行があるので、インデックスを振り直し
df = df.reset_index(drop=True)

#タイムSのデータを"00.00.0"という表記から秒数へ変換
base_time = pd.to_datetime('00.00.0', format='%M.%S.%f')
df['タイムS'] = pd.to_datetime(df['タイムS'], format='%M.%S.%f') - base_time
df['タイムS'] = df['タイムS'].dt.total_seconds()

#着差タイムが"----"の箇所を100に設定(おそらく途中棄権した?)
df = df.replace({"着差タイム" : {"----":100.0}})

#馬名でソートする
df = df.sort_values(['馬名',"年","月","日"], ascending=[True, False, False, False])
df = df.reset_index(drop=True)

In [3]:
#各種文字列をネットワークに入力するために列を追加
#それぞれの要素に該当する際にフラグを1にすることで表現
#例えば、牡の際は牡の列の値が1で、それ以外の牝、セの欄は0となる
df["牡"] = df["性別"].apply( lambda x: 1 if x == "牡" else 0 )
df["牝"] = df["性別"].apply( lambda x: 1 if x == "牝" else 0 )
df["セ"] = df["性別"].apply( lambda x: 1 if x == "セ" else 0 )
df["札幌"] = df["場所"].apply( lambda x: 1 if x == "札幌" else 0 )
df["函館"] = df["場所"].apply( lambda x: 1 if x == "函館" else 0 )
df["福島"] = df["場所"].apply( lambda x: 1 if x == "福島" else 0 )
df["新潟"] = df["場所"].apply( lambda x: 1 if x == "新潟" else 0 )
df["東京"] = df["場所"].apply( lambda x: 1 if x == "東京" else 0 )
df["中山"] = df["場所"].apply( lambda x: 1 if x == "中山" else 0 )
df["中京"] = df["場所"].apply( lambda x: 1 if x == "中京" else 0 )
df["京都"] = df["場所"].apply( lambda x: 1 if x == "京都" else 0 )
df["阪神"] = df["場所"].apply( lambda x: 1 if x == "阪神" else 0 )
df["小倉"] = df["場所"].apply( lambda x: 1 if x == "小倉" else 0 )
df["良"] = df["馬場状態"].apply( lambda x: 1 if x == "良" else 0 )
df["稍"] = df["馬場状態"].apply( lambda x: 1 if x == "稍" else 0 )
df["重"] = df["馬場状態"].apply( lambda x: 1 if x == "重" else 0 )
df["不"] = df["馬場状態"].apply( lambda x: 1 if x == "不" else 0 )
df["晴"] = df["天候"].apply( lambda x: 1 if x == "晴" else 0 )
df["曇"] = df["天候"].apply( lambda x: 1 if x == "曇" else 0 )
df["雨"] = df["天候"].apply( lambda x: 1 if x == "雨" else 0 )
df["小雨"] = df["天候"].apply( lambda x: 1 if x == "小雨" else 0 )
df["小雪"] = df["天候"].apply( lambda x: 1 if x == "小雪" else 0 )
df["雪"] = df["天候"].apply( lambda x: 1 if x == "雪" else 0 )
df["芝"] = df["トラックコード"].apply( lambda x: 1 if x == 0 else 0 )
df["ダート"] = df["トラックコード"].apply( lambda x: 1 if x == 1 else 0 )
df["障害直線芝"] = df["トラックコード"].apply( lambda x: 1 if x == 2 else 0 )
df["障害直線ダート"] = df["トラックコード"].apply( lambda x: 1 if x == 3 else 0 )
df["芝外回り"] = df["トラックコード"].apply( lambda x: 1 if x == 8 else 0 )

In [4]:
#学習データとテストデータに分割、または、確認のため少量に切り取り
#df = df.query("年 > 17")
#df = df.reset_index(drop=True)
print(len(df))

596657


In [5]:
#馬毎に過去5レースを抽出する
df_new = df.assign(年1=np.nan,月1=np.nan,日1=np.nan,馬番1=np.nan,枠番1=np.nan,年齢1=np.nan,馬体重1=np.nan,斤量1=np.nan,頭数1=np.nan,距離1=np.nan,人気1=np.nan,単勝オッズ1=np.nan,確定着順1=np.nan,タイムS1=np.nan,着差タイム1=np.nan,牡1=np.nan,牝1=np.nan,セ1=np.nan,札幌1=np.nan,函館1=np.nan,福島1=np.nan,新潟1=np.nan,東京1=np.nan,中山1=np.nan,中京1=np.nan,京都1=np.nan,阪神1=np.nan,小倉1=np.nan,良1=np.nan,稍1=np.nan,重1=np.nan,不1=np.nan,晴1=np.nan,曇1=np.nan,雨1=np.nan,小雨1=np.nan,小雪1=np.nan,雪1=np.nan,芝1=np.nan,ダート1=np.nan,障害直線芝1=np.nan,障害直線ダート1=np.nan,芝外回り1=np.nan,\
                   年2=np.nan,月2=np.nan,日2=np.nan,馬番2=np.nan,枠番2=np.nan,年齢2=np.nan,馬体重2=np.nan,斤量2=np.nan,頭数2=np.nan,距離2=np.nan,人気2=np.nan,単勝オッズ2=np.nan,確定着順2=np.nan,タイムS2=np.nan,着差タイム2=np.nan,牡2=np.nan,牝2=np.nan,セ2=np.nan,札幌2=np.nan,函館2=np.nan,福島2=np.nan,新潟2=np.nan,東京2=np.nan,中山2=np.nan,中京2=np.nan,京都2=np.nan,阪神2=np.nan,小倉2=np.nan,良2=np.nan,稍2=np.nan,重2=np.nan,不2=np.nan,晴2=np.nan,曇2=np.nan,雨2=np.nan,小雨2=np.nan,小雪2=np.nan,雪2=np.nan,芝2=np.nan,ダート2=np.nan,障害直線芝2=np.nan,障害直線ダート2=np.nan,芝外回り2=np.nan,\
                   年3=np.nan,月3=np.nan,日3=np.nan,馬番3=np.nan,枠番3=np.nan,年齢3=np.nan,馬体重3=np.nan,斤量3=np.nan,頭数3=np.nan,距離3=np.nan,人気3=np.nan,単勝オッズ3=np.nan,確定着順3=np.nan,タイムS3=np.nan,着差タイム3=np.nan,牡3=np.nan,牝3=np.nan,セ3=np.nan,札幌3=np.nan,函館3=np.nan,福島3=np.nan,新潟3=np.nan,東京3=np.nan,中山3=np.nan,中京3=np.nan,京都3=np.nan,阪神3=np.nan,小倉3=np.nan,良3=np.nan,稍3=np.nan,重3=np.nan,不3=np.nan,晴3=np.nan,曇3=np.nan,雨3=np.nan,小雨3=np.nan,小雪3=np.nan,雪3=np.nan,芝3=np.nan,ダート3=np.nan,障害直線芝3=np.nan,障害直線ダート3=np.nan,芝外回り3=np.nan,\
                   年4=np.nan,月4=np.nan,日4=np.nan,馬番4=np.nan,枠番4=np.nan,年齢4=np.nan,馬体重4=np.nan,斤量4=np.nan,頭数4=np.nan,距離4=np.nan,人気4=np.nan,単勝オッズ4=np.nan,確定着順4=np.nan,タイムS4=np.nan,着差タイム4=np.nan,牡4=np.nan,牝4=np.nan,セ4=np.nan,札幌4=np.nan,函館4=np.nan,福島4=np.nan,新潟4=np.nan,東京4=np.nan,中山4=np.nan,中京4=np.nan,京都4=np.nan,阪神4=np.nan,小倉4=np.nan,良4=np.nan,稍4=np.nan,重4=np.nan,不4=np.nan,晴4=np.nan,曇4=np.nan,雨4=np.nan,小雨4=np.nan,小雪4=np.nan,雪4=np.nan,芝4=np.nan,ダート4=np.nan,障害直線芝4=np.nan,障害直線ダート4=np.nan,芝外回り4=np.nan,\
                   年5=np.nan,月5=np.nan,日5=np.nan,馬番5=np.nan,枠番5=np.nan,年齢5=np.nan,馬体重5=np.nan,斤量5=np.nan,頭数5=np.nan,距離5=np.nan,人気5=np.nan,単勝オッズ5=np.nan,確定着順5=np.nan,タイムS5=np.nan,着差タイム5=np.nan,牡5=np.nan,牝5=np.nan,セ5=np.nan,札幌5=np.nan,函館5=np.nan,福島5=np.nan,新潟5=np.nan,東京5=np.nan,中山5=np.nan,中京5=np.nan,京都5=np.nan,阪神5=np.nan,小倉5=np.nan,良5=np.nan,稍5=np.nan,重5=np.nan,不5=np.nan,晴5=np.nan,曇5=np.nan,雨5=np.nan,小雨5=np.nan,小雪5=np.nan,雪5=np.nan,芝5=np.nan,ダート5=np.nan,障害直線芝5=np.nan,障害直線ダート5=np.nan,芝外回り5=np.nan)
datasize = len(df)
for i in range(datasize-5):
    if (i % 1000) == 0:
        print(i)
    if df['馬名'].iloc[i] == df['馬名'].iloc[i+5]:
        for j in range(5):
            df_new['年' + str(j+1)].iloc[i] = df['年'].iloc[i+j+1]
            df_new['月' + str(j+1)].iloc[i] = df['月'].iloc[i+j+1]
            df_new['日' + str(j+1)].iloc[i] = df['日'].iloc[i+j+1]
            df_new['馬番' + str(j+1)].iloc[i] = df['馬番'].iloc[i+j+1]
            df_new['枠番' + str(j+1)].iloc[i] = df['枠番'].iloc[i+j+1]
            df_new['年齢' + str(j+1)].iloc[i] = df['年齢'].iloc[i+j+1]
            df_new['馬体重' + str(j+1)].iloc[i] = df['馬体重'].iloc[i+j+1]
            df_new['斤量' + str(j+1)].iloc[i] = df['斤量'].iloc[i+j+1]
            df_new['頭数' + str(j+1)].iloc[i] = df['頭数'].iloc[i+j+1]
            df_new['距離' + str(j+1)].iloc[i] = df['距離'].iloc[i+j+1]
            df_new['人気' + str(j+1)].iloc[i] = df['人気'].iloc[i+j+1]
            df_new['単勝オッズ' + str(j+1)].iloc[i] = df['単勝オッズ'].iloc[i+j+1]
            df_new['確定着順' + str(j+1)].iloc[i] = df['確定着順'].iloc[i+j+1]
            df_new['タイムS' + str(j+1)].iloc[i] = df['タイムS'].iloc[i+j+1]
            df_new['着差タイム' + str(j+1)].iloc[i] = df['着差タイム'].iloc[i+j+1]
            df_new['牡' + str(j+1)].iloc[i] = df['牡'].iloc[i+j+1]
            df_new['牝' + str(j+1)].iloc[i] = df['牝'].iloc[i+j+1]
            df_new['セ' + str(j+1)].iloc[i] = df['セ'].iloc[i+j+1]
            df_new['札幌' + str(j+1)].iloc[i] = df['札幌'].iloc[i+j+1]
            df_new['函館' + str(j+1)].iloc[i] = df['函館'].iloc[i+j+1]
            df_new['福島' + str(j+1)].iloc[i] = df['福島'].iloc[i+j+1]
            df_new['新潟' + str(j+1)].iloc[i] = df['新潟'].iloc[i+j+1]
            df_new['東京' + str(j+1)].iloc[i] = df['東京'].iloc[i+j+1]
            df_new['中山' + str(j+1)].iloc[i] = df['中山'].iloc[i+j+1]
            df_new['中京' + str(j+1)].iloc[i] = df['中京'].iloc[i+j+1]
            df_new['京都' + str(j+1)].iloc[i] = df['京都'].iloc[i+j+1]
            df_new['阪神' + str(j+1)].iloc[i] = df['阪神'].iloc[i+j+1]
            df_new['小倉' + str(j+1)].iloc[i] = df['小倉'].iloc[i+j+1]
            df_new['良' + str(j+1)].iloc[i] = df['良'].iloc[i+j+1]
            df_new['稍' + str(j+1)].iloc[i] = df['稍'].iloc[i+j+1]
            df_new['重' + str(j+1)].iloc[i] = df['重'].iloc[i+j+1]
            df_new['不' + str(j+1)].iloc[i] = df['不'].iloc[i+j+1]
            df_new['晴' + str(j+1)].iloc[i] = df['晴'].iloc[i+j+1]
            df_new['曇' + str(j+1)].iloc[i] = df['曇'].iloc[i+j+1]
            df_new['雨' + str(j+1)].iloc[i] = df['雨'].iloc[i+j+1]
            df_new['小雨' + str(j+1)].iloc[i] = df['小雨'].iloc[i+j+1]
            df_new['小雪' + str(j+1)].iloc[i] = df['小雪'].iloc[i+j+1]
            df_new['雪' + str(j+1)].iloc[i] = df['雪'].iloc[i+j+1]
            df_new['芝' + str(j+1)].iloc[i] = df['芝'].iloc[i+j+1]
            df_new['ダート' + str(j+1)].iloc[i] = df['ダート'].iloc[i+j+1]
            df_new['障害直線芝' + str(j+1)].iloc[i] = df['障害直線芝'].iloc[i+j+1]
            df_new['障害直線ダート' + str(j+1)].iloc[i] = df['障害直線ダート'].iloc[i+j+1]
            df_new['芝外回り' + str(j+1)].iloc[i] = df['芝外回り'].iloc[i+j+1]

0


/home/uk/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
15

In [ ]:
df = df_new

#欠損値NaNが存在する行の削除
df = df.dropna()
df = df.reset_index(drop=True)

#学習には使わない項目を削除
df = df.drop(columns='馬名')
df = df.drop(columns='性別')
df = df.drop(columns='場所')
df = df.drop(columns='馬場状態')
df = df.drop(columns='天候')
df = df.drop(columns='トラックコード')

#型をfloatに変換
df = df.astype(float)

#全てfloat64になっているか確認
#pd.set_option('display.max_rows', None)
#print(df.dtypes)

In [ ]:
#保存
df.to_csv("./tmp_all.csv")